In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manyli

In [ ]:
%cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: data/val.csv            
  inflating: data/train.csv          
  inflating: data/test.csv           
  inflating: data/final_val.csv      
  inflating: data/final_test.csv     
  inflating: data/final_train.csv    


# *Importing libraries*

In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")


train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences'])

In [4]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [5]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [6]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [7]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [8]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'corrected_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvaar khol di unhone isme tumhari koi galti nhi hai',
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI'}

In [10]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'corrected_text': 'hindu rastra ke rujhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI'}

In [11]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'corrected_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [12]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['clean_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [14]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460,  10114,  15694,  10269,  13080,  10248,  13055,
          10147,  10114,  10183,  10116,  23145,  14990,  10737,  13412,  10162,
          11210, 109664,  10113,    186,  11643,  13173,  10345,  11519,    186,
          11924,    170,  10921,  10240,  12577,  10879,  10114,  13055,  34927,
          10112,  11357,  10113,  11163,  11357,  10112,    170,  10921,  10240,
          11037,  11520,  10105,  13675,  16648,  11163,  12923,  10126,  57665,
            179,  32631,  10120,  10119,  50198,  10124,  10627,  13055,  34927,
          12476,  11252,  10116,  37188,  10116,    182,  11924,  13080,    102,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [15]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', 'to', 'ku', '##ch', 'hai', 'par', 'tu', '##m', 'to', 'por', '##i', 'dun', '##iya', 'mai', 'gan', '##d', '##gi', 'mach', '##a', 'r', '##he', 'ho', 'or', 'bat', 'r', '##hi', 'b', '##ha', '##g', '##we', 'ki', 'to', 'tu', '##mh', '##e', 'ab', '##a', 'ke', 'ab', '##e', 'b', '##ha', '##g', '##wa', 'hi', 'the', 'tal', '##var', 'ke', 'dar', 'se', 'salvar', 'k', '##hol', 'di', 'un', '##hone', 'is', '##me', 'tu', '##mh', '##ari', 'ko', '##i', 'galt', '##i', 'n', '##hi', 'hai', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [16]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = train_df.iloc[0]['clean_text']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai
tokenize:              ['hin', '##du', 'to', 'ku', '##ch', 'hai', 'par', 'tu', '##m', 'to', 'por', '##i', 'dun', '##iya', 'mai', 'gan', '##d', '##gi', 'mach', '##a', 'r', '##he', 'ho', 'or', 'bat', 'r', '##hi', 'b', '##ha', '##g', '##we', 'ki', 'to', 'tu', '##mh', '##e', 'ab', '##a', 'ke', 'ab', '##e', 'b', '##ha', '##g', '##wa', 'hi', 'the', 'tal', '##var', 'ke', 'dar', 'se', 'salvar', 'k', '##hol', 'di', 'un', '##hone', 'is', '##me', 'tu', '##mh', '##ari', 'ko', '##i', 'galt', '##i', 'n', '##hi', 'hai']
convert_tokens_to_ids: [19911, 11460, 10114, 15694, 10269, 13080, 10248, 13055, 10147, 10114, 10183, 10116, 23145, 14990, 10737, 13412, 10162, 11210, 109664, 10113, 186, 11643, 13173, 10345, 11519, 186, 11924, 170, 10921, 10240, 12577, 10879, 10114, 13055, 34927, 10112, 11357

# *Calculating Weights for Labels with Imbalanced Dataset*

In [17]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [18]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [19]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [22]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [23]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k/log.jsonl"))

In [24]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.672343,0.761998,0.728723,0.761998,0.732022,0.761998,0.761998,0.592173,0.601465,0.263158,0.348525,0.921189,0.854404
2,No log,0.536487,0.795299,0.799701,0.795299,0.797313,0.795299,0.795299,0.731292,0.726528,0.607287,0.589391,0.855297,0.863666
3,No log,0.531618,0.716944,0.811844,0.716944,0.736848,0.716944,0.716944,0.755419,0.685681,0.829960,0.586552,0.680879,0.784810
4,No log,0.581644,0.775710,0.810603,0.775710,0.786466,0.775710,0.775710,0.759721,0.726800,0.728745,0.611205,0.790698,0.842395


TrainOutput(global_step=192, training_loss=0.522010087966919, metrics={'train_runtime': 465.887, 'train_samples_per_second': 65.681, 'train_steps_per_second': 1.03, 'train_loss': 0.522010087966919, 'epoch': 4.0})

In [25]:
mbert_results = trainer.evaluate()

In [26]:
print(mbert_results)

{'eval_loss': 0.5316182374954224, 'eval_accuracy': 0.7169441723800196, 'eval_precision_weighted': 0.8118438061604938, 'eval_recall_weighted': 0.7169441723800196, 'eval_f1_weighted': 0.736847635343486, 'eval_recall_micro': 0.7169441723800196, 'eval_f1_micro': 0.7169441723800196, 'eval_recall_macro': 0.7554190335708083, 'eval_f1_macro': 0.6856811720178917, 'eval_recall_positive': 0.8299595141700404, 'eval_f1_positive': 0.5865522174535049, 'eval_recall_negative': 0.6808785529715762, 'eval_f1_negative': 0.7848101265822783, 'eval_runtime': 5.6041, 'eval_samples_per_second': 182.188, 'eval_steps_per_second': 11.42, 'epoch': 4.0}


# **Hing-MBERT Test results**

In [27]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [28]:
test_results_mbert

PredictionOutput(predictions=array([[ 0.33979648, -0.83401644],
       [-0.16903734, -0.30054513],
       [ 0.6817728 , -1.3331606 ],
       ...,
       [-1.893329  ,  1.1995511 ],
       [-1.7593282 ,  1.0784913 ],
       [-0.45189884, -0.26634756]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.5160856246948242, 'test_accuracy': 0.713026444662096, 'test_precision_weighted': 0.8128921001220362, 'test_recall_weighted': 0.713026444662096, 'test_f1_weighted': 0.7333617384106561, 'test_recall_micro': 0.713026444662096, 'test_f1_micro': 0.713026444662096, 'test_recall_macro': 0.7555916475745117, 'test_f1_macro': 0.683048593783937, 'test_recall_positive': 0.8380566801619433, 'test_f1_positive': 0.5855728429985857, 'test_recall_negative': 0.6731266149870802, 'test_f1_negative': 0.7805243445692884, 'test_runtime': 5.6979, 'test_samples_per_second': 179.19, 'test_steps_per_second': 11.232})

In [29]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 1, 1, 1])

In [30]:
test_df['predicted_labels_5k_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [31]:
print(test_results_mbert.metrics)

{'test_loss': 0.5160856246948242, 'test_accuracy': 0.713026444662096, 'test_precision_weighted': 0.8128921001220362, 'test_recall_weighted': 0.713026444662096, 'test_f1_weighted': 0.7333617384106561, 'test_recall_micro': 0.713026444662096, 'test_f1_micro': 0.713026444662096, 'test_recall_macro': 0.7555916475745117, 'test_f1_macro': 0.683048593783937, 'test_recall_positive': 0.8380566801619433, 'test_f1_positive': 0.5855728429985857, 'test_recall_negative': 0.6731266149870802, 'test_f1_negative': 0.7805243445692884, 'test_runtime': 5.6979, 'test_samples_per_second': 179.19, 'test_steps_per_second': 11.232}


# **Hing BERT Training**

In [32]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")

bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['clean_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [33]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k/log.jsonl"))

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
# train the model
bert_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.689674,0.746327,0.681297,0.746327,0.685103,0.746327,0.746327,0.528084,0.508789,0.105263,0.167203,0.950904,0.850376
2,No log,0.602260,0.721841,0.763560,0.721841,0.735833,0.721841,0.721841,0.691112,0.663543,0.631579,0.523490,0.750646,0.803596
3,No log,0.536912,0.683643,0.796832,0.683643,0.706312,0.683643,0.683643,0.729320,0.655041,0.817814,0.555708,0.640827,0.754373
4,No log,0.589043,0.782566,0.797008,0.782566,0.788277,0.782566,0.782566,0.733921,0.719877,0.639676,0.587361,0.828165,0.852394
5,No log,0.898077,0.798237,0.783400,0.798237,0.786379,0.798237,0.798237,0.678098,0.694478,0.445344,0.516432,0.910853,0.872525


TrainOutput(global_step=240, training_loss=0.5166890780131023, metrics={'train_runtime': 519.5712, 'train_samples_per_second': 58.895, 'train_steps_per_second': 0.924, 'train_loss': 0.5166890780131023, 'epoch': 5.0})

In [35]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.5369120836257935, 'eval_accuracy': 0.683643486777669, 'eval_precision_weighted': 0.796831593529111, 'eval_recall_weighted': 0.683643486777669, 'eval_f1_weighted': 0.7063118346092766, 'eval_recall_micro': 0.683643486777669, 'eval_f1_micro': 0.683643486777669, 'eval_recall_macro': 0.7293203192835995, 'eval_f1_macro': 0.6550405071103184, 'eval_recall_positive': 0.8178137651821862, 'eval_f1_positive': 0.5557083906464924, 'eval_recall_negative': 0.6408268733850129, 'eval_f1_negative': 0.7543726235741444, 'eval_runtime': 5.6221, 'eval_samples_per_second': 181.605, 'eval_steps_per_second': 11.384, 'epoch': 5.0}


# **Test results for Hing-BERT**

In [36]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['predicted_labels_5k_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.5471703410148621, 'test_accuracy': 0.6787463271302644, 'test_precision_weighted': 0.8015980347713569, 'test_recall_weighted': 0.6787463271302644, 'test_f1_weighted': 0.7017902667251253, 'test_recall_micro': 0.6787463271302644, 'test_f1_micro': 0.6787463271302644, 'test_recall_macro': 0.7329818284530647, 'test_f1_macro': 0.652821895086046, 'test_recall_positive': 0.8380566801619433, 'test_f1_positive': 0.5579514824797843, 'test_recall_negative': 0.627906976744186, 'test_f1_negative': 0.7476923076923078, 'test_runtime': 5.6876, 'test_samples_per_second': 179.513, 'test_steps_per_second': 11.253}


# **Muril cased training**

In [37]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['clean_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

muril_dataset = muril_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
muril_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.688496,0.761998,0.745868,0.761998,0.751636,0.761998,0.761998,0.639035,0.648600,0.400810,0.448980,0.877261,0.848220
2,No log,0.650659,0.761019,0.743938,0.761019,0.749913,0.761019,0.761019,0.635633,0.645401,0.392713,0.442922,0.878553,0.847880
3,0.678200,0.631945,0.781587,0.764465,0.781587,0.768982,0.781587,0.781587,0.656090,0.669836,0.412955,0.477752,0.899225,0.861920
4,0.678200,0.591080,0.778648,0.782398,0.778648,0.780408,0.778648,0.778648,0.706527,0.703112,0.566802,0.553360,0.846253,0.852865
5,0.678200,0.610213,0.765916,0.779283,0.765916,0.771455,0.765916,0.765916,0.707778,0.696606,0.595142,0.551595,0.820413,0.841617
6,0.567600,0.616653,0.731636,0.775365,0.731636,0.745645,0.731636,0.731636,0.708599,0.677286,0.663968,0.544850,0.753230,0.809722


TrainOutput(global_step=1152, training_loss=0.6018900871276855, metrics={'train_runtime': 757.7345, 'train_samples_per_second': 40.384, 'train_steps_per_second': 2.534, 'train_loss': 0.6018900871276855, 'epoch': 6.0})

In [39]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.5910802483558655, 'eval_accuracy': 0.7786483839373164, 'eval_precision_weighted': 0.7823981137545883, 'eval_recall_weighted': 0.7786483839373164, 'eval_f1_weighted': 0.7804084519071824, 'eval_recall_micro': 0.7786483839373164, 'eval_f1_micro': 0.7786483839373164, 'eval_recall_macro': 0.7065274247036792, 'eval_f1_macro': 0.7031121335638999, 'eval_recall_positive': 0.5668016194331984, 'eval_f1_positive': 0.5533596837944664, 'eval_recall_negative': 0.8462532299741602, 'eval_f1_negative': 0.8528645833333334, 'eval_runtime': 5.4147, 'eval_samples_per_second': 188.561, 'eval_steps_per_second': 11.82, 'epoch': 6.0}


In [40]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['predicted_labels_5k_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

{'test_loss': 0.5944593548774719, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.766694476082377, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.7619213432113588, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.6874483465670735, 'test_f1_macro': 0.6809524351719365, 'test_recall_positive': 0.5506072874493927, 'test_f1_positive': 0.5240847784200385, 'test_recall_negative': 0.8242894056847545, 'test_f1_negative': 0.8378200919238346, 'test_runtime': 5.5539, 'test_samples_per_second': 183.836, 'test_steps_per_second': 11.524}


In [41]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['clean_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
import torch
torch.cuda.empty_cache()

In [43]:
xlmr_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.682672,0.758080,0.696030,0.758080,0.655592,0.758080,0.758080,0.501378,0.435137,0.004049,0.008032,0.998708,0.862242
2,No log,0.600760,0.746327,0.773111,0.746327,0.756169,0.746327,0.746327,0.703127,0.683119,0.619433,0.541593,0.786822,0.824645
3,0.663900,0.678922,0.773751,0.772821,0.773751,0.773279,0.773751,0.773751,0.689514,0.690298,0.526316,0.529532,0.852713,0.851064
4,0.663900,0.655331,0.721841,0.749702,0.721841,0.732516,0.721841,0.721841,0.669060,0.652151,0.566802,0.496454,0.771318,0.807848


TrainOutput(global_step=768, training_loss=0.6316839456558228, metrics={'train_runtime': 441.0337, 'train_samples_per_second': 69.382, 'train_steps_per_second': 4.353, 'train_loss': 0.6316839456558228, 'epoch': 4.0})

In [44]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.6007598042488098, 'eval_accuracy': 0.7463271302644466, 'eval_precision_weighted': 0.7731106990984227, 'eval_recall_weighted': 0.7463271302644466, 'eval_f1_weighted': 0.7561687882899328, 'eval_recall_micro': 0.7463271302644466, 'eval_f1_micro': 0.7463271302644466, 'eval_recall_macro': 0.7031274519034616, 'eval_f1_macro': 0.6831187350585077, 'eval_recall_positive': 0.6194331983805668, 'eval_f1_positive': 0.5415929203539823, 'eval_recall_negative': 0.7868217054263565, 'eval_f1_negative': 0.8246445497630333, 'eval_runtime': 5.4354, 'eval_samples_per_second': 187.843, 'eval_steps_per_second': 11.775, 'epoch': 4.0}


In [45]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['predicted_labels_5k_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.6163123250007629, 'test_accuracy': 0.7159647404505387, 'test_precision_weighted': 0.7554191767257925, 'test_recall_weighted': 0.7159647404505387, 'test_f1_weighted': 0.72968309841281, 'test_recall_micro': 0.7159647404505387, 'test_f1_micro': 0.7159647404505387, 'test_recall_macro': 0.678966722112377, 'test_f1_macro': 0.6543750291824253, 'test_recall_positive': 0.6072874493927125, 'test_f1_positive': 0.5084745762711865, 'test_recall_negative': 0.7506459948320413, 'test_f1_negative': 0.800275482093664, 'test_runtime': 5.489, 'test_samples_per_second': 186.008, 'test_steps_per_second': 11.66}


In [46]:
test_df.to_csv('final_test_5k.csv',index=False, encoding='utf-8')